In [1]:
import pandas as pd

In [2]:
df_lottery = pd.read_csv('data.csv')

In [3]:
df_lottery.head()

,name,Remarks
0,Cambrian Heights School,Livingston Community only
1,Connaught School,NaN
2,Copperfield School,NaN
3,Cranston School,NaN
4,Crossing Park School,NaN


In [4]:
df_lottery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     57 non-null     object
 1   Remarks  11 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


In [27]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('../database.db')



In [ ]:
# Load the school data from the database into a DataFrame
df_school = pd.read_sql_query("SELECT * FROM schools", conn)
conn.close()

As the lottery data is also from CBE, which is the same source of schools table, the format of 'name' is the same. No cleaning is necessary. Can proceed to matching directly.

In [8]:
def check_school_id(ranking_school):
    for row in df_school.itertuples():
        
        if ranking_school in row.name:
            return row.school_id
    return None

df_lottery['school_id'] = df_lottery['name'].apply(check_school_id)

In [9]:
df_lottery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       57 non-null     object
 1   Remarks    11 non-null     object
 2   school_id  57 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.5+ KB


In [15]:
df_lottery['school_id'].duplicated().any()

False

Matched perfectly.

Add a new column 'school_year'

In [19]:
df_lottery['school_year'] = '2024 - 2025​'

In [20]:
df_lottery.head()

,name,Remarks,school_id,school_year
0,Cambrian Heights School,Livingston Community only,31,2024 - 2025​
1,Connaught School,NaN,14,2024 - 2025​
2,Copperfield School,NaN,244,2024 - 2025​
3,Cranston School,NaN,216,2024 - 2025​
4,Crossing Park School,NaN,188,2024 - 2025​


# Database insertion

In [24]:
data_tuples = []

for i, row in enumerate(df_lottery.itertuples(), 1):
    data_tuple = (
        i,
        row.school_id,
        row.school_year,
        row.Remarks,
        )
    data_tuples.append(data_tuple)

In [28]:
cur = conn.cursor()

cur.execute("""
            CREATE TABLE IF NOT EXISTS school_lottery (
            id INTEGER PRIMARY KEY,
            school_id INTEGER,
            school_year TEXT,
            remarks TEXT,
            FOREIGN KEY(school_id) REFERENCES schools(school_id)
            )
            """)

In [29]:
# use executemany to insert the data into the database
cur.executemany('''
    INSERT INTO school_lottery VALUES (?,?,?,?)
''', data_tuples)


In [30]:
conn.commit()

In [31]:
conn.close()